## Loick CUER et Léa FELDMAN

## Natural Language Processing : Information Retrieval Challenge

### Importations des librairies principales

In [27]:
import urllib.request as url_request
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
import re
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score
from rank_bm25 import BM25Okapi
nltk.download('all')
from collections import defaultdict, Counter
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams
import random
from nltk import ne_chunk, pos_tag
import math

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\loick\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\loick\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\loick\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\loick\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\loick\AppData\Roaming\nltk_data...
[

### Chargement de NFCorpus

In [2]:
def loadNFCorpus(): #load the NFCorpus
    
	dir = "../data/raw/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename, 'r', encoding='utf-8') as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename, 'r', encoding='utf-8') as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename, 'r', encoding='utf-8') as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc

In [3]:
dicDoc, dicReq, dicReqDoc = loadNFCorpus()

print("Contenu de dicDoc (5 premiers éléments) :") #affichage des 5 premiers éléments de dicDoc, chaque élément contient l'ID du document et son contenu.
for i, (key, value) in enumerate(dicDoc.items()):
    if i >= 5:
        break
    print(f"Document ID: {key}")
    print(f"Contenu: {value[:]}...")
    print()

Contenu de dicDoc (5 premiers éléments) :
Document ID: MED-118
Contenu: alkylphenols human milk relations dietary habits central taiwan pubmed ncbi abstract aims study determine concentrations num nonylphenol np num octylphenol op num human milk samples examine related factors including mothers demographics dietary habits women consumed median amount cooking oil significantly higher op concentrations num ng/g consumed num ng/g num op concentration significantly consumption cooking oil beta num num fish oil capsules beta num num adjustment age body mass index bmi np concentration significantly consumption fish oil capsules beta num num processed fish products beta num num food pattern cooking oil processed meat products factor analysis strongly op concentration human milk num determinations aid suggesting foods consumption nursing mothers order protect infants np/op exposure num elsevier rights reserved 
...

Document ID: MED-329
Contenu: phosphate vascular toxin pubmed ncbi abstract el

## BM25 basique

In [6]:
def text2TokenList(text): #fonction qui prend en entrée un texte et renvoie une liste de mots sans les stopwords
	stopword = set(stopwords.words('english'))
	#print("LEN DE STOPWORD=",len(stopword))
	word_tokens = word_tokenize(text.lower())
	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>2]
	return word_tokens_without_stops

def run_bm25_only(startDoc,endDoc):

	dicDoc, dicReq, dicReqDoc = loadNFCorpus()

	docsToKeep=[]
	reqsToKeep=[]
	dicReqDocToKeep=defaultdict(dict)

	ndcgTop=10

	i=startDoc
	for reqId in dicReqDoc:
		if i > (endDoc - startDoc) :  #nbDocsToKeep:
			break
		for docId in dicReqDoc[reqId]:
			dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
			docsToKeep.append(docId)
			i = i + 1
		reqsToKeep.append(reqId)
	docsToKeep = list(set(docsToKeep))

	#"""
	allVocab ={}
	for k in docsToKeep:
		docTokenList = text2TokenList(dicDoc[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListDoc = list(allVocab)
	#print("doc vocab=",len(allVocabListDoc))
	allVocab ={}
	for k in reqsToKeep:
		docTokenList = text2TokenList(dicReq[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListReq = list(allVocab)

	corpusDocTokenList = []
	corpusReqTokenList = {}
	corpusDocName=[]
	corpusDicoDocName={}
	i = 0
	for k in docsToKeep:
		docTokenList = text2TokenList(dicDoc[k])
		corpusDocTokenList.append(docTokenList)
		corpusDocName.append(k)
		corpusDicoDocName[k] = i
		i = i + 1

	#print("reqs...")
	corpusReqName=[]
	corpusDicoReqName={}
	i = 0
	for k in reqsToKeep:
		reqTokenList = text2TokenList(dicReq[k])
		corpusReqTokenList[k] = reqTokenList
		corpusReqName.append(k)
		corpusDicoReqName[k] = i
		i = i + 1

	#print("bm25 doc indexing...")
	bm25 = BM25Okapi(corpusDocTokenList)

	ndcgCumul=0
	corpusReqVec={}
	ndcgBM25Cumul=0
	nbReq=0

	for req in corpusReqTokenList:
		j=0
		reqTokenList = corpusReqTokenList[req]
		doc_scores = bm25.get_scores(reqTokenList)
		trueDocs = np.zeros(len(corpusDocTokenList))

		for docId in corpusDicoDocName:
			if req in dicReqDocToKeep:
				if docId in dicReqDocToKeep[req]:
					#get position docId
					posDocId = corpusDicoDocName[docId]
					trueDocs[posDocId] = dicReqDocToKeep[req][docId]
					#print("TOKEEP=",docId)
					#print(trueDocs)
		ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
		nbReq = nbReq + 1
	ndcgBM25Cumul = ndcgBM25Cumul / nbReq
	return ndcgBM25Cumul
first_doc_key = next(iter(dicDoc))  #Let's take a look at the first document of our corpus and the output of our pre-processing applicated on this document
first_doc_value = dicDoc[first_doc_key]

print(f"Clé du premier document : {first_doc_key}")
print(f"Contenu du premier document : {first_doc_value}")
print(text2TokenList(first_doc_value))

nb_docs = 150
print(f"Résultat en prenant seulement {nb_docs} documents en compte : {run_bm25_only(0, nb_docs)}")

Clé du premier document : MED-118
Contenu du premier document : alkylphenols human milk relations dietary habits central taiwan pubmed ncbi abstract aims study determine concentrations num nonylphenol np num octylphenol op num human milk samples examine related factors including mothers demographics dietary habits women consumed median amount cooking oil significantly higher op concentrations num ng/g consumed num ng/g num op concentration significantly consumption cooking oil beta num num fish oil capsules beta num num adjustment age body mass index bmi np concentration significantly consumption fish oil capsules beta num num processed fish products beta num num food pattern cooking oil processed meat products factor analysis strongly op concentration human milk num determinations aid suggesting foods consumption nursing mothers order protect infants np/op exposure num elsevier rights reserved 

['alkylphenols', 'human', 'milk', 'relations', 'dietary', 'habits', 'central', 'taiwan', '

In [13]:
def run_fullbm25_only():
    dicDoc, dicReq, dicReqDoc = loadNFCorpus()

    ndcgTop = 10

    corpusDocTokenList = []
    corpusDocName = []
    corpusDicoDocName = {}

    # Prétraitement de tous les documents
    for i, (docId, docText) in enumerate(dicDoc.items()):
        docTokenList = text2TokenList(docText, True, True)
        corpusDocTokenList.append(docTokenList)
        corpusDocName.append(docId)
        corpusDicoDocName[docId] = i

    # Prétraitement de toutes les requêtes
    corpusReqTokenList = {}
    for reqId, reqText in dicReq.items():
        corpusReqTokenList[reqId] = text2TokenList(reqText, True, True)

    # Création de l'index BM25
    bm25 = BM25Okapi(corpusDocTokenList)

    ndcgBM25Cumul = 0
    nbReq = 0

    for reqId, reqTokenList in corpusReqTokenList.items():
        doc_scores = bm25.get_scores(reqTokenList)
        trueDocs = np.zeros(len(corpusDocTokenList))

        if reqId in dicReqDoc:
            for docId, relevance in dicReqDoc[reqId].items():
                if docId in corpusDicoDocName:
                    posDocId = corpusDicoDocName[docId]
                    trueDocs[posDocId] = relevance

        # Tri des documents par score décroissant
        sorted_indices = np.argsort(doc_scores)[::-1]
        sorted_scores = doc_scores[sorted_indices]
        sorted_true_docs = trueDocs[sorted_indices]

        ndcgBM25Cumul += ndcg_score([sorted_true_docs], [sorted_scores], k=ndcgTop)
        nbReq += 1

    ndcgBM25Average = ndcgBM25Cumul / nbReq
    return ndcgBM25Average

# Utilisation de la fonction
result = run_fullbm25_only()
print(f"NDCG@10 moyen sur l'ensemble du corpus : {result}")

NDCG@10 moyen sur l'ensemble du corpus : 0.3735768567999993


On constate qu'en utilisant tout le corpus au lieu des 150 premiers documents, le score ndcg est bien plus bas. Cependant cela ne veut pas dire qu le modèle est moins performant. On a icic un score plus réaliste des performances du systeme avec un corpus plus large. + de documents = plus de bruits et plus de difficulté à placer les documents pertinents dans les 10 premières positions. Nous allons continuer d'utiliser bm25 avec 150 documents pour une execution du code plus rapide mais évaluer sur l'ensemble du ccorpus serait plus exact.

### Pre-Processing alternatifs

La fonction qui se charge du pre-processing dans le code de base est text2TokenList. Apportons-y quelques changements.

In [8]:
def text2TokenList2(text, use_bigrams=True, use_trigrams=True):
    stopword = set(stopwords.words('english')) #load the list of english stopwords
    text = re.sub(r'[^a-zA-Z]', ' ', text) # delete characters that are not letters
    text = text.lower() #convert words into lowercase
    word_tokens = word_tokenize(text) # tokenize our sentences into tokens (words)
    word_tokens_filtered = [word for word in word_tokens if word not in stopword and len(word) > 2] #filter stopwords and short words
    lemmatizer = WordNetLemmatizer() #lemmatize our tokens
    word_tokens_lemmatized = [lemmatizer.lemmatize(token) for token in word_tokens_filtered]
    if use_bigrams: #add bigrams
        bigrams = list(nltk.bigrams(word_tokens_filtered))
        word_tokens_lemmatized.extend([' '.join(bigram) for bigram in bigrams])
    if use_trigrams: #add trigrams
        trigrams = list(nltk.trigrams(word_tokens_filtered))
        word_tokens_lemmatized.extend([' '.join(trigram) for trigram in trigrams])
    return word_tokens_lemmatized

def run_bm25_only2(startDoc,endDoc):

	dicDoc, dicReq, dicReqDoc = loadNFCorpus()

	docsToKeep=[]
	reqsToKeep=[]
	dicReqDocToKeep=defaultdict(dict)

	ndcgTop=10

	i=startDoc
	for reqId in dicReqDoc:
		if i > (endDoc - startDoc) :  #nbDocsToKeep:
			break
		for docId in dicReqDoc[reqId]:
			dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
			docsToKeep.append(docId)
			i = i + 1
		reqsToKeep.append(reqId)
	docsToKeep = list(set(docsToKeep))

	#"""
	allVocab ={}
	for k in docsToKeep:
		docTokenList = text2TokenList2(dicDoc[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListDoc = list(allVocab)
	#print("doc vocab=",len(allVocabListDoc))
	allVocab ={}
	for k in reqsToKeep:
		docTokenList = text2TokenList2(dicReq[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListReq = list(allVocab)

	corpusDocTokenList = []
	corpusReqTokenList = {}
	corpusDocName=[]
	corpusDicoDocName={}
	i = 0
	for k in docsToKeep:
		docTokenList = text2TokenList2(dicDoc[k])
		corpusDocTokenList.append(docTokenList)
		corpusDocName.append(k)
		corpusDicoDocName[k] = i
		i = i + 1

	#print("reqs...")
	corpusReqName=[]
	corpusDicoReqName={}
	i = 0
	for k in reqsToKeep:
		reqTokenList = text2TokenList2(dicReq[k])
		corpusReqTokenList[k] = reqTokenList
		corpusReqName.append(k)
		corpusDicoReqName[k] = i
		i = i + 1

	#print("bm25 doc indexing...")
	bm25 = BM25Okapi(corpusDocTokenList)

	ndcgCumul=0
	corpusReqVec={}
	ndcgBM25Cumul=0
	nbReq=0

	for req in corpusReqTokenList:
		j=0
		reqTokenList = corpusReqTokenList[req]
		doc_scores = bm25.get_scores(reqTokenList)
		trueDocs = np.zeros(len(corpusDocTokenList))

		for docId in corpusDicoDocName:
			if req in dicReqDocToKeep:
				if docId in dicReqDocToKeep[req]:
					#get position docId
					posDocId = corpusDicoDocName[docId]
					trueDocs[posDocId] = dicReqDocToKeep[req][docId]
					#print("TOKEEP=",docId)
					#print(trueDocs)
		ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
		nbReq = nbReq + 1
	ndcgBM25Cumul = ndcgBM25Cumul / nbReq
	return ndcgBM25Cumul

first_doc_key = next(iter(dicDoc))  #Let's take a look at the first document of our corpus and the output of our pre-processing applicated on this document
first_doc_value = dicDoc[first_doc_key]

print(f"Clé du premier document : {first_doc_key}")
print(f"Contenu du premier document : {first_doc_value}")
print(text2TokenList2(first_doc_value, use_bigrams=True, use_trigrams=True))


nb_docs = 150
print(f"Résultat en prenant seulement {nb_docs} documents en compte : {run_bm25_only2(0, nb_docs)}")



Clé du premier document : MED-118
Contenu du premier document : alkylphenols human milk relations dietary habits central taiwan pubmed ncbi abstract aims study determine concentrations num nonylphenol np num octylphenol op num human milk samples examine related factors including mothers demographics dietary habits women consumed median amount cooking oil significantly higher op concentrations num ng/g consumed num ng/g num op concentration significantly consumption cooking oil beta num num fish oil capsules beta num num adjustment age body mass index bmi np concentration significantly consumption fish oil capsules beta num num processed fish products beta num num food pattern cooking oil processed meat products factor analysis strongly op concentration human milk num determinations aid suggesting foods consumption nursing mothers order protect infants np/op exposure num elsevier rights reserved 

['alkylphenols', 'human', 'milk', 'relation', 'dietary', 'habit', 'central', 'taiwan', 'pu

ici, nous avons amélioré la fonction de tokenisation en intégrant la lemmatisation, la suppression de caractères non alphabétiques, et l'ajout d'options pour inclure des bigrammes et trigrammes dans le vocabulaire. Ces ajustements permettent de mieux capturer le sens et les relations entre les mots, enrichissant ainsi la représentation des documents et des requêtes pour optimiser les performances de BM25.

On constate que de  nombreux mots parasites sont tokenisés comme "num" ou "op", on va donc les filtrer en choississant de ne conservere que les mots de 4 lettres et +

In [11]:
def text2TokenList3(text, use_bigrams=True, use_trigrams=True):
    stopword = set(stopwords.words('english')) #load the list of english stopwords
    text = re.sub(r'[^a-zA-Z]', ' ', text) # delete characters that are not letters
    text = text.lower() #convert words into lowercase
    word_tokens = word_tokenize(text) # tokenize our sentences into tokens (words)
    word_tokens_filtered = [word for word in word_tokens if word not in stopword and len(word) > 3] #filter stopwords and words of 3 letters or less
    lemmatizer = WordNetLemmatizer() #lemmatize our tokens
    word_tokens_lemmatized = [lemmatizer.lemmatize(token) for token in word_tokens_filtered]
    if use_bigrams: #add bigrams
        bigrams = list(nltk.bigrams(word_tokens_filtered))
        word_tokens_lemmatized.extend(['_'.join(bigram) for bigram in bigrams])
    if use_trigrams: #add trigrams
        trigrams = list(nltk.trigrams(word_tokens_filtered))
        word_tokens_lemmatized.extend(['_'.join(trigram) for trigram in trigrams])
    return word_tokens_lemmatized

def run_bm25_only3(startDoc,endDoc):

	dicDoc, dicReq, dicReqDoc = loadNFCorpus()

	docsToKeep=[]
	reqsToKeep=[]
	dicReqDocToKeep=defaultdict(dict)

	ndcgTop=10

	i=startDoc
	for reqId in dicReqDoc:
		if i > (endDoc - startDoc) :  #nbDocsToKeep:
			break
		for docId in dicReqDoc[reqId]:
			dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
			docsToKeep.append(docId)
			i = i + 1
		reqsToKeep.append(reqId)
	docsToKeep = list(set(docsToKeep))

	#"""
	allVocab ={}
	for k in docsToKeep:
		docTokenList = text2TokenList3(dicDoc[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListDoc = list(allVocab)
	#print("doc vocab=",len(allVocabListDoc))
	allVocab ={}
	for k in reqsToKeep:
		docTokenList = text2TokenList3(dicReq[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListReq = list(allVocab)

	corpusDocTokenList = []
	corpusReqTokenList = {}
	corpusDocName=[]
	corpusDicoDocName={}
	i = 0
	for k in docsToKeep:
		docTokenList = text2TokenList3(dicDoc[k])
		corpusDocTokenList.append(docTokenList)
		corpusDocName.append(k)
		corpusDicoDocName[k] = i
		i = i + 1

	#print("reqs...")
	corpusReqName=[]
	corpusDicoReqName={}
	i = 0
	for k in reqsToKeep:
		reqTokenList = text2TokenList3(dicReq[k])
		corpusReqTokenList[k] = reqTokenList
		corpusReqName.append(k)
		corpusDicoReqName[k] = i
		i = i + 1

	#print("bm25 doc indexing...")
	bm25 = BM25Okapi(corpusDocTokenList)

	ndcgCumul=0
	corpusReqVec={}
	ndcgBM25Cumul=0
	nbReq=0

	for req in corpusReqTokenList:
		j=0
		reqTokenList = corpusReqTokenList[req]
		doc_scores = bm25.get_scores(reqTokenList)
		trueDocs = np.zeros(len(corpusDocTokenList))

		for docId in corpusDicoDocName:
			if req in dicReqDocToKeep:
				if docId in dicReqDocToKeep[req]:
					#get position docId
					posDocId = corpusDicoDocName[docId]
					trueDocs[posDocId] = dicReqDocToKeep[req][docId]
					#print("TOKEEP=",docId)
					#print(trueDocs)
		ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
		nbReq = nbReq + 1
	ndcgBM25Cumul = ndcgBM25Cumul / nbReq
	return ndcgBM25Cumul

first_doc_key = next(iter(dicDoc))  #Let's take a look at the first document of our corpus and the output of our pre-processing applicated on this document
first_doc_value = dicDoc[first_doc_key]

print(f"Clé du premier document : {first_doc_key}")
print(f"Contenu du premier document : {first_doc_value}")
print(text2TokenList3(first_doc_value, use_bigrams=True, use_trigrams=True))


nb_docs = 150
print(f"Résultat en prenant seulement {nb_docs} documents en compte : {run_bm25_only3(0, nb_docs)}")

Clé du premier document : MED-118
Contenu du premier document : alkylphenols human milk relations dietary habits central taiwan pubmed ncbi abstract aims study determine concentrations num nonylphenol np num octylphenol op num human milk samples examine related factors including mothers demographics dietary habits women consumed median amount cooking oil significantly higher op concentrations num ng/g consumed num ng/g num op concentration significantly consumption cooking oil beta num num fish oil capsules beta num num adjustment age body mass index bmi np concentration significantly consumption fish oil capsules beta num num processed fish products beta num num food pattern cooking oil processed meat products factor analysis strongly op concentration human milk num determinations aid suggesting foods consumption nursing mothers order protect infants np/op exposure num elsevier rights reserved 

['alkylphenols', 'human', 'milk', 'relation', 'dietary', 'habit', 'central', 'taiwan', 'pu

In [15]:
def text2TokenList4(text, use_bigrams=True, use_trigrams=True): #add inteligent n-grams
    # Charger la liste des stopwords anglais
    stopword = set(stopwords.words('english'))
    
    # Prétraitement du texte
    text = re.sub(r'[^\w\s]', ' ', text)  # Conserver les chiffres, remplacer la ponctuation par des espaces
    text = text.lower()
    
    # Tokenisation
    word_tokens = word_tokenize(text)
    
    # Filtrage des stopwords et des mots courts
    word_tokens_filtered = [word for word in word_tokens if word not in stopword and len(word) > 2]
    
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    word_tokens_lemmatized = [lemmatizer.lemmatize(token) for token in word_tokens_filtered]
    
    # Ajout des n-grammes pertinents
    if use_bigrams or use_trigrams:
        n_grams = []
        if use_bigrams:
            n_grams.extend(ngrams(word_tokens_filtered, 2))
        if use_trigrams:
            n_grams.extend(ngrams(word_tokens_filtered, 3))
        
        # Filtrer les n-grammes pertinents (par exemple, ceux qui apparaissent plus d'une fois)
        n_gram_freq = nltk.FreqDist(n_grams)
        relevant_n_grams = [ng for ng, freq in n_gram_freq.items() if freq > 1]
        
        # Ajouter les n-grammes pertinents à la liste des tokens
        word_tokens_lemmatized.extend(['_'.join(ng) for ng in relevant_n_grams])
    
    return word_tokens_lemmatized

def run_bm25_only4(startDoc,endDoc):

	dicDoc, dicReq, dicReqDoc = loadNFCorpus()

	docsToKeep=[]
	reqsToKeep=[]
	dicReqDocToKeep=defaultdict(dict)

	ndcgTop=10

	i=startDoc
	for reqId in dicReqDoc:
		if i > (endDoc - startDoc) :  #nbDocsToKeep:
			break
		for docId in dicReqDoc[reqId]:
			dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
			docsToKeep.append(docId)
			i = i + 1
		reqsToKeep.append(reqId)
	docsToKeep = list(set(docsToKeep))

	#"""
	allVocab ={}
	for k in docsToKeep:
		docTokenList = text2TokenList4(dicDoc[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListDoc = list(allVocab)
	#print("doc vocab=",len(allVocabListDoc))
	allVocab ={}
	for k in reqsToKeep:
		docTokenList = text2TokenList4(dicReq[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListReq = list(allVocab)

	corpusDocTokenList = []
	corpusReqTokenList = {}
	corpusDocName=[]
	corpusDicoDocName={}
	i = 0
	for k in docsToKeep:
		docTokenList = text2TokenList4(dicDoc[k])
		corpusDocTokenList.append(docTokenList)
		corpusDocName.append(k)
		corpusDicoDocName[k] = i
		i = i + 1

	#print("reqs...")
	corpusReqName=[]
	corpusDicoReqName={}
	i = 0
	for k in reqsToKeep:
		reqTokenList = text2TokenList4(dicReq[k])
		corpusReqTokenList[k] = reqTokenList
		corpusReqName.append(k)
		corpusDicoReqName[k] = i
		i = i + 1

	#print("bm25 doc indexing...")
	bm25 = BM25Okapi(corpusDocTokenList)

	ndcgCumul=0
	corpusReqVec={}
	ndcgBM25Cumul=0
	nbReq=0

	for req in corpusReqTokenList:
		j=0
		reqTokenList = corpusReqTokenList[req]
		doc_scores = bm25.get_scores(reqTokenList)
		trueDocs = np.zeros(len(corpusDocTokenList))

		for docId in corpusDicoDocName:
			if req in dicReqDocToKeep:
				if docId in dicReqDocToKeep[req]:
					#get position docId
					posDocId = corpusDicoDocName[docId]
					trueDocs[posDocId] = dicReqDocToKeep[req][docId]
					#print("TOKEEP=",docId)
					#print(trueDocs)
		ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
		nbReq = nbReq + 1
	ndcgBM25Cumul = ndcgBM25Cumul / nbReq
	return ndcgBM25Cumul

first_doc_key = next(iter(dicDoc))  #Let's take a look at the first document of our corpus and the output of our pre-processing applicated on this document
first_doc_value = dicDoc[first_doc_key]

print(f"Clé du premier document : {first_doc_key}")
print(f"Contenu du premier document : {first_doc_value}")
print(text2TokenList4(first_doc_value, use_bigrams=True, use_trigrams=True))


nb_docs = 150
print(f"Résultat en prenant seulement {nb_docs} documents en compte : {run_bm25_only4(0, nb_docs)}")

Clé du premier document : MED-118
Contenu du premier document : alkylphenols human milk relations dietary habits central taiwan pubmed ncbi abstract aims study determine concentrations num nonylphenol np num octylphenol op num human milk samples examine related factors including mothers demographics dietary habits women consumed median amount cooking oil significantly higher op concentrations num ng/g consumed num ng/g num op concentration significantly consumption cooking oil beta num num fish oil capsules beta num num adjustment age body mass index bmi np concentration significantly consumption fish oil capsules beta num num processed fish products beta num num food pattern cooking oil processed meat products factor analysis strongly op concentration human milk num determinations aid suggesting foods consumption nursing mothers order protect infants np/op exposure num elsevier rights reserved 

['alkylphenols', 'human', 'milk', 'relation', 'dietary', 'habit', 'central', 'taiwan', 'pu

gridsearch for bm25 hyperparameters

In [30]:
def run_bm25_with_params(startDoc, endDoc, k1, b):
    dicDoc, dicReq, dicReqDoc = loadNFCorpus()

    docsToKeep = []
    reqsToKeep = []
    dicReqDocToKeep = defaultdict(dict)

    ndcgTop = 10

    i = startDoc
    for reqId in dicReqDoc:
        if i > (endDoc - startDoc):
            break
        for docId in dicReqDoc[reqId]:
            dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
            docsToKeep.append(docId)
            i += 1
        reqsToKeep.append(reqId)
    docsToKeep = list(set(docsToKeep))

    corpusDocTokenList = []
    corpusReqTokenList = {}
    corpusDocName = []
    corpusDicoDocName = {}

    for i, k in enumerate(docsToKeep):
        docTokenList = text2TokenList4(dicDoc[k])
        corpusDocTokenList.append(docTokenList)
        corpusDocName.append(k)
        corpusDicoDocName[k] = i

    for k in reqsToKeep:
        reqTokenList = text2TokenList4(dicReq[k])
        corpusReqTokenList[k] = reqTokenList

    bm25 = BM25Okapi(corpusDocTokenList, k1=k1, b=b)

    ndcgBM25Cumul = 0
    nbReq = 0

    for req in corpusReqTokenList:
        reqTokenList = corpusReqTokenList[req]
        doc_scores = bm25.get_scores(reqTokenList)
        trueDocs = np.zeros(len(corpusDocTokenList))

        if req in dicReqDocToKeep:
            for docId, relevance in dicReqDocToKeep[req].items():
                if docId in corpusDicoDocName:
                    posDocId = corpusDicoDocName[docId]
                    trueDocs[posDocId] = relevance

        ndcgBM25Cumul += ndcg_score([trueDocs], [doc_scores], k=ndcgTop)
        nbReq += 1

    ndcgBM25Cumul /= nbReq
    return ndcgBM25Cumul

# Définir les valeurs à tester pour k1 et b
k1_values = [0.5, 1.0, 1.2, 1.5, 2.0]
b_values = [0.5, 0.6, 0.75, 0.8, 0.9]

best_score = 0
best_params = None

nb_docs = 150  # Nombre de documents à considérer

for k1 in k1_values:
    for b in b_values:
        score = run_bm25_with_params(0, nb_docs, k1, b)
        if score > best_score:
            best_score = score
            best_params = (k1, b)

print(f"Meilleurs paramètres : k1={best_params[0]}, b={best_params[1]}")

# Exécuter avec les meilleurs paramètres
final_score = run_bm25_with_params(0, nb_docs, best_params[0], best_params[1])
print(f"Score final avec les meilleurs paramètres : {final_score}")

Meilleurs paramètres : k1=1.5, b=0.9
Score final avec les meilleurs paramètres : 0.7848648682406477


### Creation du modèle word2vec

In [9]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [10]:
tokenized_corpus = [text2TokenList(doc, use_bigrams=True, use_trigrams=True) for doc in list(dicDoc.values())]

Le code charge le corpus NFCorpus et définit la fonction text2TokenList, qui prétraite et tokenise le texte en supprimant les caractères non alphabétiques, en lemmatizant les mots, et en ajoutant des bigrammes et trigrammes si demandé. Ensuite, il applique cette fonction à tous les documents du corpus pour créer une liste tokenisée, enrichissant ainsi la représentation des documents pour une utilisation ultérieure dans des tâches de traitement de texte ou d'analyse.

In [11]:
model = Word2Vec(tokenized_corpus, vector_size=400, window=8, min_count=3, workers=6, sg=0, epochs=100) #create our word2vec model
bm25 = BM25Okapi(tokenized_corpus) #apply BM25 to our tokenized corpus

In [14]:
def retrieve_documents_bm25_w2v(query, bm25, model, tokenized_corpus, top_n=5):
    query_tokens = text2TokenList(query)
    bm25_scores = bm25.get_scores(query_tokens)
    
    # Calculer la similarité Word2Vec
    query_vec = np.mean([model.wv[word] for word in query_tokens if word in model.wv], axis=0)
    w2v_scores = np.array([np.dot(query_vec, np.mean([model.wv[word] for word in doc if word in model.wv], axis=0)) for doc in tokenized_corpus])
    
    # Normaliser les scores
    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores))
    w2v_scores = (w2v_scores - np.min(w2v_scores)) / (np.max(w2v_scores) - np.min(w2v_scores))
    
    # Combiner les scores (vous pouvez ajuster les poids)
    combined_scores = 0.7 * bm25_scores + 0.3 * w2v_scores
    return np.argsort(-combined_scores)[:top_n]

def evaluate_retrieval_bm25(dicReq, dicReqDoc, bm25, doc_ids, model, tokenized_corpus, top_n=5):
    ndcg_scores = []
    for req_id, query in dicReq.items():
        true_scores_dict = dicReqDoc[req_id]
        retrieved_doc_indices = retrieve_documents_bm25_w2v(query, bm25, model, tokenized_corpus, top_n=top_n)
        retrieved_doc_ids = [doc_ids[idx] for idx in retrieved_doc_indices]
        retrieved_scores = np.array([true_scores_dict.get(doc_id, 0) for doc_id in retrieved_doc_ids], dtype=np.float64)
        ideal_scores_list = sorted(true_scores_dict.values(), reverse=True)[:len(retrieved_scores)]
        ideal_scores_list.extend([0] * (top_n - len(ideal_scores_list)))
        ideal_scores = np.array(ideal_scores_list, dtype=np.float64)
        ndcg = ndcg_score([ideal_scores], [retrieved_scores], k=top_n)
        ndcg_scores.append(ndcg)
    return np.mean(ndcg_scores)

doc_ids = list(dicDoc.keys())
bm25_ndcg = evaluate_retrieval_bm25(dicReq, dicReqDoc, bm25, doc_ids, model, tokenized_corpus)
print("Résultat du modèle BM25 avec pre-processing et word2vec : "+str(bm25_ndcg))

Résultat du modèle BM25 avec pre-processing et word2vec : 0.9480280287435777
